Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
let name = ""
let rollno = ""

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Monads

In this section, we will extend the state monad example from class to implement OCaml-like references. The idea is to use the functional heaps that we briefly saw at the beginning of the lambda calculus lectures and use this as the state used in the state monad. The definition of functional heap is below:

In [ ]:
module type FHEAP = sig
  type ('k,'v) t
  val empty_heap : ('k,'v) t
  val set        : ('k,'v) t -> 'k -> 'v -> ('k,'v) t
  val get        : ('k,'v) t -> 'k -> 'v option
end

module FHeap : FHEAP = struct
  type ('k,'v) t = 'k -> 'v option
  let empty_heap = fun k -> None
  let set h x v = fun k -> if k = x then Some v else h k
  let get h x = h x
end

Here is the monad signature

In [ ]:
module type MONAD = sig
  type 'a t
  val return : 'a -> 'a t
  val (>>=)  : 'a t -> ('a -> 'b t) -> 'b t
end

## Multiple References of the same type

First we will start with a monad that allows creation of new reference cells, read and update them but restricted to storing the same type of value. You will implement the `Ref_monad` module with the following signature:

In [ ]:
module type REF_MONAD = sig
  type value   (* type of value *)
  type ref     (* type of reference *)
  include MONAD
  val mk_ref : value -> ref t
  val (!) : ref -> value t
  val (:=) : ref -> value -> unit t
  val run_state : 'a t -> 'a
end

Notice that `mk_ref` creates "fresh" reference cells. These reference cells are different from every other reference cell that has already been created or will be created in this monad. This property is known as **generativity**. We had seen a similar concept when we defined `fresh` for substitutions in the lambda calculus assignement (assignment 2). 

We can simulate freshness by using a counter value `c`, whose state is also maintained as part of the state. Everytime you want to create a fresh reference, update this counter `c`. We will use this counter to index into the functional heap. Hence, the reference itself (`ref` type) is just an integer. 

## Problem 1: Implement the Ref_monad

In [ ]:
module Ref_monad (V : sig type t end)
  : REF_MONAD with type value = V.t = struct
  type value = V.t
  type ref = int (* just an index into the FHeap.t *)
  type 'a t = 
    int (* counter *) * (int, value) FHeap.t (* heap indexed by ref *) -> 
    int (* new counter *) * (int, value) FHeap.t (* new heap *) * 'a 
  
  (* Implement the missing functions *)
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")
end

In [ ]:

module M = struct
  module R = Ref_monad(struct type t = int end)
  open R
  let program = 
    mk_ref 10 >>= fun i -> 
    !i
  ;; 
  assert (run_state program = 10)
end


In [ ]:
module M = struct
  module R = Ref_monad(struct type t = int end)
  open R
  let program = 
    mk_ref 10 >>= fun i ->
    !i >>= fun iv ->
    (i := iv + 1) >>= fun () ->
    !i
  ;; 
  assert (run_state program = 11)
end


In [ ]:
module M = struct
  module R = Ref_monad(struct type t = int end)
  open R
  let program = 
    mk_ref 10 >>= fun i ->
    mk_ref 20 >>= fun j ->
    !i >>= fun iv ->
    (i := iv + 1) >>= fun () ->
    !j >>= fun jv ->
    !i >>= fun iv ->
    return (iv + jv)
  ;;

  assert(run_state program = 31)
end


## Polymorphic Refences

Let us now graduate to implementing references of different types. It turns out that storing different typed values in a key-value map and retrieving it back is tricky; every `value` that is put into a given map must be the same. Luckily, we have a concept called universal types that provides a way to safely go from arbitrary types to the universal type and back. Here is an implementation of universal type in OCaml:

In [ ]:
module Univ : sig
  type t
  type 'a packer = {pack : 'a -> t; unpack : t -> 'a option}
  val mk : unit -> 'a packer
end = struct
  type t = exn
  type 'a packer = {pack : 'a -> t; unpack : t -> 'a option}
  let mk : type a. unit -> a packer = fun () ->
    let module M () = struct exception E of a end in
    let module F = M () in
    {pack = (fun v -> F.E v); 
     unpack = fun p -> match p with F.E v -> Some v | _ -> None}
end

The implementation details are not important. The `mk` function returns a packer for an arbitrary type, which includes an function to pack values of this arbitrary type to the universal type and unpack such a type. Here is how you might use the functions.

In [ ]:
module M = struct
  include Univ
  let int_packer = mk ()
  let float_packer = mk ()
  
  let l = [int_packer.pack 10; float_packer.pack 20.25]
  
  let rec get packer l =
    match l with
    | [] -> None
    | x::xs -> 
        match packer.unpack x with
        | Some v -> Some v
        | None -> get packer xs
end;;

M.get M.int_packer M.l;;
M.get M.float_packer M.l


The important thing to notice above is the type of `get` function. Given a `'a packer` and a list of universal values, the function retrieves the value packed with the given packer.

## Problem 2

Use the universal type to implement a polymorphic reference cell monad with the following signature. 

In [ ]:
module type POLY_REF_MONAD = sig
  type 'a ref (* type of reference *)
  include MONAD
  val mk_ref : 'a -> 'a ref t
  val (!) : 'a ref -> 'a t
  val (:=) : 'a ref -> 'a -> unit t
  val run_state : 'a t -> 'a
end

Observe in the following that the reference cell is not just an integer but also includes the packer. Use this packer to pack the values before inserting into the heap and unpack after retrieving from the heap.

In [ ]:
module Poly_ref_monad : POLY_REF_MONAD = struct
  type 'a ref = int * 'a Univ.packer
  type 'a t = int * (int, Univ.t) FHeap.t -> int * (int, Univ.t) FHeap.t * 'a

  (* Implement the missing functions *)

(* YOUR CODE HERE *)
 raise (Failure "Not implemented")
end

In [ ]:
module M = struct
  open Poly_ref_monad

  let program = 
    mk_ref 10 >>= fun i ->
    mk_ref "20" >>= fun s ->
    return ()
  ;;
  
  assert (run_state program = ())
end


In [ ]:
module M = struct
  open Poly_ref_monad

  let program = 
    mk_ref 10 >>= fun i ->
    mk_ref "20" >>= fun s ->
    !i >>= fun iv ->
    (i := iv + 1) >>= fun () ->
    !s >>= fun sv ->
    !i >>= fun iv ->
    return (iv + int_of_string sv)
  ;;
  assert (run_state program = 31)
end


# GADTs

In [ ]:
type z = Z
type 'n s = S : 'n -> 'n s

## Cross product

A cross product of two lists is given by the function:

```ocaml
cross : 'a list -> 'b list -> ('a,'b) list
```

which returns a new list with all possible pairs of the elements of the two lists. The implementation of the function in OCaml is:

In [ ]:
(* scalar product of value with a list *)
let rec cross_v_l v l =
  match l with
  | [] -> []
  | x::xs -> (v,x)::cross_v_l v xs
  
let rec cross l1 l2 = 
  match l1 with
  | [] -> []
  | x::xs -> (cross_v_l x l2) @ cross xs l2

Here are some examples:

In [ ]:
cross [1;2] ["a";"b";"c";"d"];;
cross [] [];;
cross [] [1];;
cross [1] ["a"];;

Naturally, if `l1` is of length `n` and `l2` is of length `m`, then the resultant list is of length `n * m`. Let us enforce this property in a length-indexed list.

In [ ]:
(* Length-indexed list *)
type (_,_) list =
  | Nil  : ('a, z ) list
  | Cons : 'a * ('a,'n) list -> ('a, 'n s) list

## Problem 3

The scalar product of a value with a list of values does not change the length of the list. Implement the function 

```ocaml
cross_v_l : 'a -> ('b, n) list -> ('a * 'b, n) list
```

In [ ]:
let cross_v_l v l = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (cross_v_l 1 (Cons ("a", Cons ("b", Nil))) = Cons ((1, "a"), Cons ((1, "b"), Nil)));

## Appending lists 

Observe in `cross` that we have used `append` function. For reference, the append function on regular list is:

In [ ]:
let rec append l1 l2 =
  match l1 with
  | [] -> l2 
  | x::xs -> x::append xs l2

We need a corresponding implementation of append for the length-indexed case. Since the length of the result of the append of two lists is the sum of the lengths of the two lists, we need carry around proofs for the sum of length as we had seen in the lecture for tail recursive reverse of a list. In particular, we will show that:

\\[
\begin{array}{c}
0 + n = n \\
m+n = o \implies (m+1)+n = o+1
\end{array}
\\]

The two axioms above will correspond to the two cases in the body of the append.

## Problem 4

Complete the implementation of the following definition of `plus` which correspond to the theorems above

In [ ]:
(* uncomment and implement

type (_,_,_) plus =
  | PlusZero  : (z,?,?) plus
  | PlusSucc : (?, ?, 'o) plus -> (?, ?, 'o s) plus
  
*)

(* YOUR CODE HERE *)
 raise (Failure "Not implemented")

In [ ]:
let zero_plus_zero_eq_zero : (z,z,z) plus = PlusZero
let two_plus_three_eq_five : (z s s, z s s s, z s s s s s) plus = PlusSucc (PlusSucc PlusZero)

## Problem 5

Implement the append for length indexed lists of length `m` and `n`. The first argument is the proof that `m + n = o`.

In [ ]:
let rec append : type m n o. (m,n,o) plus -> ('a, m) list -> ('a, n) list -> ('a, o) list =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (append zero_plus_zero_eq_zero Nil Nil = Nil);;
assert (append two_plus_three_eq_five (Cons (1,Cons(2,Nil))) (Cons (3, Cons(4,Cons(5,Nil)))) 
        = Cons (1,Cons(2,Cons(3,Cons(4,Cons(5,Nil))))))
        

## Multiplication

Let's consider the implementation of `cross`:

```ocaml
let rec cross l1 l2 = 
  match l1 with
  | [] -> []
  | x::xs -> (cross_v_l x l2) @ cross xs l2
```

Given a list `l1` of length `m` and `l2` of length `n`, the result will be a list of length `m * n`. We do not have multiplication on type-level church numerals. Hence, we will implement them ourselves. 

Let's consider the empty case (first case) in `cross`. If `l1` is empty, then the length of the result list is empty whatever the length of `l2` is. This corresponds to the axiom 

\\[
0 * n = 0 \quad (ax1)
\\]

Now focus on the non-empty case in `cross`. There is a call to `append` (`@`), a call to `cross_v_l` and a recursive call to `cross`. For the corresponding imlementation using length-indexed lists, `cross_v_l` does not need an explicit proof as you have implemented earlier. We know that `cross_v_l x l2` will be of length `n` if `l2` is of length `n`. But we need proofs for `append` and the recursive call of `cross`. 

Given that `l1` is a non-empty list in the non-empty case, we can represent its length as `m s`. That is to say that the length of `l1` is non-zero. Then `xs` is of length `m`. The recursive call on `cross` is now on lists of length `m` and `n`. Let us say that `m * n = p`. We know that the length of the result of `cross_v_l x l2` call is `n`. Hence, `append` is applied on lists of lengths `n` and `p`. Let us say that `n + p = o`.

Putting all the facts together:

\\[
n + p = o ~\wedge~ m * n = p \implies (m+1) * n = o \quad (ax2)
\\]

This can be shown to be true using the arithmetic axioms of distributivity and multiplicative identity. The axioms $ax1$ and $ax2$ are precisely what we need for implementing `cross` for length-indexed lists.

## Problem 6

Complete the definition of `mult` type below based on the axioms $ax1$ and $ax2$.

In [ ]:
(* UNCOMMENT AND IMPLEMENT

type (_,_,_) mult =
  | MultZero : (?, ?, ?) mult
  | MultSucc : (?, ?, ?) plus * (?, ?, ?) mult -> ('m s, 'n, 'o) mult
  
*)

(* YOUR CODE HERE *)
 raise (Failure "Not implemented")

In [ ]:
let zero_mult_two_eq_zero : (z, z s s, z) mult = MultZero;;
let one_mult_two_eq_two : (z s, z s s, z s s) mult = MultSucc (PlusSucc (PlusSucc PlusZero), MultZero);;

## Problem 7

In the following, write down terms (proofs) which have those types (theorems).

In [ ]:
let two_mult_two_eq_four : (z s s, z s s, z s s s s) mult =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

let two_mult_one_eq_two : (z s s, z s, z s s) mult =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
let two_mult_two_eq_four : (z s s, z s s, z s s s s) mult = two_mult_two_eq_four
let two_mult_one_eq_two : (z s s, z s, z s s) mult = two_mult_one_eq_two

## Problem 8

Implement cross which accepts the multiplication proof as the first argument.

In [ ]:
let rec cross : type m n o. (m,n,o) mult -> ('a,m) list -> ('b,n) list -> ('a * 'b, o) list =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (cross zero_mult_two_eq_zero Nil (Cons (1,Cons(2,Nil))) = Nil);;
assert (cross one_mult_two_eq_two (Cons("a", Nil)) (Cons (1,Cons(2,Nil))) =
        Cons(("a",1), Cons(("a",2), Nil)));;

# Zipping

You will now implement zip functions that will zip length indexed lists.

## Problem 9

First, implement zip function that works on the same lengthed lists. Compared to the standard library functions such as `map2` and `iter2`, which raises an `Invalid_argument` exception when the lists are of different lengths, your `zip` function will statically reject lists of different lengths. 

In [ ]:
let rec zip : type n. ('a,n) list -> ('b,n) list -> ('a *'b, n) list =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (zip (Cons(1,Cons(2,Nil))) (Cons("a",Cons("b",Nil))) = 
        Cons((1,"a"), Cons((2,"b"), Nil)));;

## Zipping lists of unequal length

Let's say we want to allow zipping lists of unequal lengths such that we only zip together the first `n` elements where `n` is the smaller of the lengths of the two lists. For example,

```ocaml
let rec zip_matching l1 l2 =
  match l1,l2 with
  | [],_ -> []
  | _,[] -> []
  | x::xs,y::ys -> (x,y)::zip_matching xs ys
;;
```

```ocaml
assert (zip_matching [1;2] [1.0;2.0;3.0] = [(1,1.0);(2;2.0)])
```

The extra element in the second list are ignored. If the lists are of lengths, `m` and `n`, then the result list is of length `o` where `o` is the minimum of `m` and `n`. We do not have minimum function on type level-numerals. Hence, we will implement them. 

## Problem 10

Implement the `min` type that corresponds to the three cases in the body of the zip_matching function

In [ ]:
(* UNCOMMENT AND IMPLEMENT

type (_,_,_) min =
  | MinZero1 : ? 
  | MinZero2 : ? 
  | MinSucc  : ?
*)

(* YOUR CODE HERE *)
 raise (Failure "Not implemented")

In [ ]:
let min_zero_four_zero : (z, z s s s s, z) min = MinZero1;;

## Problem 11

In the following, write down terms (proofs) which have those types (theorems).

In [ ]:
let min_three_zero_zero : (z s s s, z, z) min = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

let min_three_five_eq_three : (z s s s, z s s s s s, z s s s) min = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
let min_three_zero_zero : (z s s s, z, z) min = min_three_zero_zero

## Problem 12

Implement the `zip_matching` function on length-indexed list:

In [ ]:
let rec zip_matching : type n m o. (n,m,o) min -> ('a, n) list -> ('b, m) list -> ('a * 'b, o) list =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (zip_matching min_zero_four_zero Nil (Cons(1,(Cons(2,Cons(3,Cons(4,Nil)))))) = Nil);;
assert (zip_matching min_three_zero_zero  (Cons(1,(Cons(2,Cons(3,Nil))))) Nil = Nil);;

# Streams

Consider the standard lazy stream definition.

In [ ]:
type 'a stream = Cons of 'a * 'a stream Lazy.t

let hd (Cons(x,_)) = x
let tl (Cons(x,xs)) = Lazy.force xs
let rec take n s =
  if n = 0 then []
  else hd s::(take (n-1) (tl s))

## Problem 13

Define a value `pow2 : int stream` whose elements are the powers of two: `<1; 2; 4; 8; 16, ...>`

In [ ]:
let pow2 = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (List.hd (List.rev (take 10 pow2)) = 512);;

## Problem 14

Define a function `interleave : 'a stream -> 'a stream -> 'a stream`, such that `interleave <a1; a2; a3; ...> <b1; b2; b3; ...>` is the stream `<a1; b1; a2; b2; a3; b3; ...>`. For example, `interleave nats pow2` would be `<0; 1; 1; 2; 2; 4; 3; 8; ...>`.

In [ ]:
let rec interleave s1 s2 = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
let rec zeros = Cons(0,lazy zeros)
let rec ones = Cons(1,lazy ones);;
assert (take 10 (interleave zeros ones) = [0; 1; 0; 1; 0; 1; 0; 1; 0; 1]);;

## Approximately e

The exponential function  $e^x$  can be computed by the following infinite sum:

\\[
e^x = \frac{x^0}{0!} + \frac{x^1}{1!} + \frac{x^2}{2!} + \ldots + \frac{x^k}{k!} + \ldots
\\]

## Problem 15

Define a function `e_terms : float -> float stream`. Element `k` of the stream should be term `k` from the infinite sum. For example, `e_terms 1.0` is the stream `<1.0; 1.0; 0.5; 0.1666...; 0.041666...; ...>`. The easy way to compute that involves a function that computes $f(k)=\frac{x^k}{k!}$. 

In [ ]:
let e_terms x = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (List.hd (List.rev (take 10 (e_terms 1.0))) = 2.75573192239858925e-06);;

## Problem 16

Define a function `total : float stream -> float stream`, such that `total <a; b; c; ...>` is a running total of the input elements, i.e., `<a; a+.b; a+.b+.c; ...>`.

In [ ]:
let total = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
let rec f x = Cons (x, lazy (f (x+.1.0))) in
assert (take 10 (total (f 1.0)) = [1.; 3.; 6.; 10.; 15.; 21.; 28.; 36.; 45.; 55.]);;

## Problem 17

Define a function `within : float -> float stream -> float`, such that `within eps s` is the first element of `s` for which the absolute difference between that element and the element before it is strictly less than `eps`. If there is no such element, within is permitted not to terminate (i.e., go into an "infinite loop"). As a precondition, the tolerance `eps` must be strictly positive. For example, `within 0.1 <1.0; 2.0; 2.5; 2.75; 2.875; 2.9375; 2.96875; ...>` is `2.9375`.

In [ ]:
let rec within eps s =
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (within 1e-15 (total (e_terms 1.0)) = 2.71828182845904553);;

## Problem 18

Finally, define a function `e : float -> float -> float` such that `e x eps` is  $e^x$  computed to within a tolerance of `eps`, which must be strictly positive. Note that there is an interesting boundary case where `x=1.0` for the first two terms of the sum; you could choose to drop the first term (which is always 1.0) from the stream before using `within`.

In [ ]:
let e x eps = 
  (* YOUR CODE HERE *)
   raise (Failure "Not implemented")

In [ ]:
assert (e 1.0 1e-15 = 2.71828182845904553);;